In [3]:
#Imports
import json
import nltk
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import re  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from cleantext import *

In [4]:
#Load models Sentiment
nltk.download('vader_lexicon')
nltk.download('punkt')
sentimentIntensityAnalyzer = SentimentIntensityAnalyzer()

#Load models Stance

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/annafernandezrajal/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/annafernandezrajal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#Settings
pd.set_option('display.width', 1000)

In [14]:
#Load data
twitter_data = pd.read_csv('dataset/train.csv', encoding="unicode_escape", lineterminator='\r')

twitter_test = pd.read_csv('dataset/test.csv', encoding="unicode_escape", lineterminator='\r')
twitter_data.head(10)

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos
5,Hillary can not win. Here's hoping the Dems of...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
6,Respect FOR the law and respect BY the law Yes...,Hillary Clinton,NONE,2. The tweet does NOT expresses opinion about ...,pos
7,I don't want to be appointed to an Ambassador ...,Hillary Clinton,NONE,2. The tweet does NOT expresses opinion about ...,neg
8,#StopHillary2016 @HillaryClinton if there was ...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
9,@HillaryClinton End lawless #ClintonFoundation...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [15]:
#Calculate Sentiment
def calculateSentiment(text):
    return sentimentIntensityAnalyzer.polarity_scores(text)['compound']


def sentiment(dataframe ,results):
    result = []
    for value in results:
        if value > 0.61:
            result.append("pos")
        else:
            result.append("neg")
    dataframe['sentimentVader'] = result

results = twitter_data['Tweet'].apply(calculateSentiment)
sentiment(twitter_data, results)


print("Accuracy score: ", accuracy_score(twitter_data['Sentiment'] , twitter_data['sentimentVader'] ))

display(twitter_data)

Accuracy score:  0.664378860672615


,Tweet,Target,Stance,Opinion Towards,Sentiment,sentimentVader
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos,neg
...,...,...,...,...,...,...
2909,"There's a law protecting unborn eagles, but no...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,neg
2910,I am 1 in 3... I have had an abortion #Abortio...,Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,other,neg
2911,How dare you say my sexual preference is a cho...,Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,neg,neg
2912,"Equal rights for those 'born that way', no rig...",Legalization of Abortion,AGAINST,2. The tweet does NOT expresses opinion about ...,neg,neg


In [16]:
# Calculate Stance Detection

training_pre = pd.read_csv('dataset/train.csv', encoding="unicode_escape", lineterminator='\r')
test_pre = pd.read_csv('dataset/test.csv', encoding="unicode_escape", lineterminator='\r')


def tfidf(training, test):
    training['text_clean'] = clean_text(training, 'Tweet')
    test['text_clean'] = clean_text(test, 'Tweet')

    all_text = training['text_clean'].values.tolist() + test['text_clean'].values.tolist()
    vocab = flatten_words(all_text, get_unique=True)
    tfidf = TfidfVectorizer(stop_words='english', vocabulary=vocab)
    training_matrix = tfidf.fit_transform(training.text_clean)
    test_matrix = tfidf.fit_transform(test.text_clean)
    training.drop('Target', inplace=True, axis=1)
    label_train = training["Stance"] 
    training.drop('Stance', inplace=True, axis=1)
    training.drop('Opinion Towards', inplace=True, axis=1)
    training.drop('Sentiment', inplace=True, axis=1)
    test.drop('Target', inplace=True, axis=1)
    label_test = test["Stance"] 
    test.drop('Stance', inplace=True, axis=1)
    test.drop('Opinion Towards', inplace=True, axis=1)
    test.drop('Sentiment', inplace=True, axis=1)

    training = features(training)
    training = pd.concat([training, pd.DataFrame(training_matrix.todense())], axis=1)

    test = features(test)
    test = pd.concat([test, pd.DataFrame(test_matrix.todense())], axis=1)

    return training, test, label_train, label_test


#Pre-processinf TF-IDF

training, test, label_train, label_test = tfidf(training_pre, test_pre)


#svm= SVC(kernel = "poly", degree=4)
svm = LinearSVC(dual=False, max_iter=5000)
features = training.columns[2:]
X = training[features].values

svm.fit(X, label_train)

features_tr = training[features].values
predicted = svm.predict(features_tr)
print("Accuracy score: ", accuracy_score(label_train, predicted))


twitter_data["StanceSVM"] = predicted
twitter_data.head()


TypeError: 'Index' object is not callable

In [ ]:
## TESTING

#Sentiment
results = twitter_test['Tweet'].apply(calculateSentiment)
sentiment(twitter_test, results)

#Stance
features_dev = test[features].values
dev_predicted = svm.predict(features_dev)
twitter_test["StanceSVM"] = dev_predicted


sentiment_score = accuracy_score(twitter_test["Sentiment"], twitter_test["sentimentVader"])
print("Sentiment Accuracy on test:", sentiment_score)

stance_score = accuracy_score(twitter_test["Stance"], twitter_test["StanceSVM"])
print("Stance Accuracy on test:", stance_score)

In [ ]:
twitter_test.head()

In [ ]:
twitter_data.to_csv('output.csv')
print("saved")